In [ ]:
import pandas as pd
import numpy as np

In [ ]:

!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 22.1 MB/s eta 0:00:00


In [ ]:
import json
with open(r"/content/train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]},
 {'context': 'The story tells about t

In [ ]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs


In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [ ]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,

    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args,use_cuda=False
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!rm -rf outputs

In [ ]:

# Train the model
model.train_model(train, eval_data=test)

convert squad examples to features:   0%|          | 0/9 [00:00<?, ?it/s]Could not find answer: 'The Final Empire,The Well of Ascension, and The Hero of Ages.' vs. 'The Final Empire, The Well of Ascension, and The Hero of Ages'
Could not find answer: 'king of the' vs. 'Sakura'
Could not find answer: 'Sakura. They were friends from' vs. 'to look after thier kingdom.'
Could not find answer: 'in the attic of the crumbling village library. Little did she know that this book held the key to' vs. 'Little did she know that this book held the key to a magical adventure beyond her wildest dreams.'
Could not find answer: 'library. Little did she know that this book held the key to a magical adventure beyond her wildest dreams. As she opened the book, the words shimmered and danced on' vs. 'As she opened the book, the words shimmered and danced on the pages, transporting Amelia to a mystical realm filled with mythical creatures and hidden treasures.'
add example index and unique id: 100%|████████

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

correct,▁▁▁▁▁
eval_loss,█▇▄▂▁
global_step,▁▃▅▆█
incorrect,▁▁▁▁▁
similar,▁▁▁▁▁
train_loss,██▅▂▁
correct,0
eval_loss,-0.59522
global_step,5
incorrect,2
similar,1


Running Epoch 0 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 0, 0, 0, 0],
  'similar': [2, 2, 2, 3, 3],
  'incorrect': [1, 1, 1, 0, 0],
  'train_loss': [4.794121742248535,
   4.772407531738281,
   4.335729598999023,
   3.936760425567627,
   3.6800549030303955],
  'eval_loss': [0.17871181666851044,
   0.11016296595335007,
   0.06700263172388077,
   0.03779533877968788,
   0.022318676114082336]})

In [ ]:

# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]


In [ ]:

answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2786.91it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['and', 'and skill.']}]


In [ ]:
def calculate_accuracy(predictions, ground_truths):
    """
   To Calculate the accuracy of predictions by comparing them to the ground truth answers.

    Args:
        predictions (list): List of predicted answers.
        ground_truths (list): List of ground truth answers.

    Returns:
        float: Accuracy of the predictions.
    """
    correct = 0
    total = len(predictions)

    for pred, gt in zip(predictions, ground_truths):
        if pred == gt:
            correct += 1

    accuracy = correct / total
    return accuracy

def calculate_em(predictions, ground_truths):
    """
    

    Args:
        predictions (list): List of predicted answers.
        ground_truths (list): List of ground truth answers.

    Returns:
        float: Exact Match (EM) score of the predictions.
    """
    em = 0

    for pred, gt in zip(predictions, ground_truths):
        if pred == gt:
            em += 1

    em_score = em / len(predictions)
    return em_score

def calculate_f1(predictions, ground_truths):
    """
   To calculate the F1 score for the predictions by comparing them to the ground truth answers.

    Args:
        predictions (list): List of predicted answers.
        ground_truths (list): List of ground truth answers.

    Returns:
        float: F1 score of the predictions.
    """
    f1_scores = []

    for pred, gt in zip(predictions, ground_truths):
        common_tokens = set(pred) & set(gt)
        precision = len(common_tokens) / len(pred) if len(pred) > 0 else 0
        recall = len(common_tokens) / len(gt) if len(gt) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

    avg_f1 = sum(f1_scores) / len(f1_scores)
    return avg_f1


In [1]:
# predictions = ['a Mistborn of', 'is a Mistborn of','is a Mistborn of power and skill']
# ground_truths = ['is a Mistborn of']

# # Calculate metrics
# accuracy = calculate_accuracy(predictions, ground_truths)
# em_score = calculate_em(predictions, ground_truths)
# f1_score = calculate_f1(predictions, ground_truths)

# # Print results
# print("Accuracy:", accuracy)
# print("EM Score:", em_score)
# print("F1 Score:", f1_score)